# Introduction

In this demo we will be using the [MovieLens Dataset](https://grouplens.org/datasets/movielens/). The dataset slicing options are:

* **[Small](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip):** 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.
* **[Full](http://files.grouplens.org/datasets/movielens/ml-latest.zip):** 27,000,000 ratings and 1,100,000 tag applications applied to 58,000 movies by 280,000 users. Last updated 9/2018.



In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -q --show-progress
!unzip ml-latest-small.zip
!rm ml-latest-small.zip

ml-latest-small.zip 100%[===================>] 955.28K  1.26MB/s    in 0.7s    
Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [2]:
%env DATASET_PATH /home/jovyan/labs/lab3-hive/ml-latest-small

env: DATASET_PATH=/home/jovyan/labs/lab3-hive/ml-latest-small


# Creating a database

We will be using Hive's [Beeline CLS](https://cwiki.apache.org/confluence/display/Hive/HiveServer2+Clients#HiveServer2Clients-Beeline%E2%80%93CommandLineShell).

Listing databases:

In [3]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "SHOW DATABASES;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
+----------------+
| database_name  |
+----------------+
| default        |
| movielens      |
+----------------+
2 rows selected (1.482 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [4]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "CREATE DATABASE IF NOT EXISTS movielens;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.288 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [5]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "SHOW DATABASES;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
+----------------+
| database_name  |
+----------------+
| default        |
| movielens      |
+----------------+
2 rows selected (1.387 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


## Creating a table ratings in the movielens database

In [6]:
#Verifying tables
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; SHOW tables;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.284 seconds)
OK
+-----------+
| tab_name  |
+-----------+
+-----------+
No rows selected (0.247 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [7]:
#Creating table ratings
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; CREATE TABLE ratings(                                           \
                                                                     userID  INT,                              \
                                                                     movieID INT,                              \
                                                                     rating  INT,                              \
                                                                     time    INT )                             \
                                                            ROW FORMAT DELIMITED                               \
                                                            FIELDS TERMINATED BY ','                           \
                                                            STORED AS TEXTFILE                                 \
                                                            tblproperties(\"skip.header.line.count\"=\"1\");"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.267 seconds)
OK
No rows affected (0.905 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [8]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; SHOW tables;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.233 seconds)
OK
+-----------+
| tab_name  |
+-----------+
| ratings   |
+-----------+
1 row selected (0.434 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


## Import data from local file system

In [9]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; SELECT * FROM ratings;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.24 seconds)
OK
+-----------------+------------------+-----------------+---------------+
| ratings.userid  | ratings.movieid  | ratings.rating  | ratings.time  |
+-----------------+------------------+-----------------+---------------+
+-----------------+------------------+-----------------+---------------+
No rows selected (1.964 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [10]:
#Loading data
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens;  LOAD DATA LOCAL INPATH '$DATASET_PATH/ratings.csv'\
                                                OVERWRITE INTO TABLE ratings;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.248 seconds)
Loading data to table movielens.ratings
OK
No rows affected (1.423 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [11]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; SELECT * FROM ratings LIMIT 10;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.204 seconds)
OK
+-----------------+------------------+-----------------+---------------+
| ratings.userid  | ratings.movieid  | ratings.rating  | ratings.time  |
+-----------------+------------------+-----------------+---------------+
| 1               | 1                | 4               | 964982703     |
| 1               | 3                | 4               | 964981247     |
| 1               | 6                | 4               | 964982224     |
| 1               | 47               | 5               | 964983815     |
| 1               | 50               | 5               | 964982931     |
| 1               | 70               | 3               | 964982400     |
| 1               | 101              | 5               | 964980868     |
| 1               | 110              | 4               | 964982176     |
| 1

## Verifying the file created by Hive

In [12]:
!hdfs dfs -ls /user/hive/warehouse/
!hdfs dfs -ls /user/hive/warehouse/movielens.db/
!hdfs dfs -ls /user/hive/warehouse/movielens.db/ratings

Found 1 items
drwxrwxr-x   - jovyan supergroup          0 2020-10-14 00:08 /user/hive/warehouse/movielens.db
Found 1 items
drwxrwxr-x   - jovyan supergroup          0 2020-10-14 00:09 /user/hive/warehouse/movielens.db/ratings
Found 1 items
-rwxrwxr-x   1 jovyan supergroup    2483723 2020-10-14 00:09 /user/hive/warehouse/movielens.db/ratings/ratings.csv


## Finding the most popular movie

In [13]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; SELECT movieID, COUNT(movieID) as ratingCount  \
                                                                                FROM ratings                   \
                                                                                GROUP BY movieID               \
                                                                                ORDER BY ratingCount DESC      \
                                                                                LIMIT 10;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.232 seconds)
Query ID = jovyan_20201014001001_cff76090-e09b-4f5f-aa55-5a50fc341b5c
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1602633141551_0001, Tracking URL = http://jupyter-thedatasociety-2dlab-2dhadoop-2dbyh3hjxu:8088/proxy/application_1602633141551_0001/
Kill Command = /home/jovyan/resources/local/hadoop-2.9.2/bin/hadoop job  -kill job_1602633141551_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-1

## Finding the name of the most popular movie

### Creating a new table that contains movies' title

In [14]:
#Creating a new table called movieNames
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens; CREATE TABLE movieNames(                                                   \
                                                                        movieID  INT,                                     \
                                                                        title STRING)                                     \
                                                                        ROW FORMAT DELIMITED                              \
                                                                        FIELDS TERMINATED BY ','                          \
                                                                        STORED AS TEXTFILE                                \
                                                                        tblproperties(\"skip.header.line.count\"=\"1\");"                                                        

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.213 seconds)
OK
No rows affected (0.806 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


In [15]:
#Loading data into movieNames table
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens;  LOAD DATA LOCAL INPATH '$DATASET_PATH/movies.csv'       \
                                                OVERWRITE INTO TABLE movieNames;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.952 seconds)
Loading data to table movielens.movienames
OK
No rows affected (1.342 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


### Creating a view to store the movies' popularity

In [16]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens;  CREATE VIEW topMoviesIds AS                                          \
                                                SELECT movieID, COUNT(movieID) as ratingCount        \
                                                FROM ratings                                         \
                                                GROUP BY movieID                                     \
                                                ORDER BY ratingCount DESC;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.273 seconds)
OK
No rows affected (2.271 seconds)
Beeline version 2.3.7 by Apache Hive
Closing: 0: jdbc:hive2://


### Finding the name of the most popular movie

In [17]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens;  SELECT n.title, ratingCount                                                          \
                                                FROM topMoviesIds t JOIN movieNames n ON t.movieID = n.movieID       \
                                                LIMIT 10;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.249 seconds)
Query ID = jovyan_20201014001243_a9eb7f6c-3929-4d4a-97a3-4da75e05fe70
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1602633141551_0003, Tracking URL = http://jupyter-thedatasociety-2dlab-2dhadoop-2dbyh3hjxu:8088/proxy/application_1602633141551_0003/
Kill Command = /home/jovyan/resources/local/hadoop-2.9.2/bin/hadoop job  -kill job_1602633141551_0003
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-1

# It's your turn

## Find the movie with the highest average time

In [ ]:
!cd ~/resources/local/hive-2.3.7/bin/ && ./beeline -u "jdbc:hive2://" -e "USE movielens;  CREATE VIEW avgMoviesIds AS\
SELECT movieID, AVG(rating) as ratingAVG\
FROM ratings\
GROUP BY movieID\
ORDER BY ratingAVG DESC;\
SELECT n.title as titles, a.ratingAVG as average, t.ratingCount as rating\
FROM avgMoviesIds a JOIN topMoviesIds t ON a.movieID = t.movieID\
JOIN movieNames n ON t.movieID = n.movieID\
WHERE t.ratingCount >10\
ORDER BY average DESC\
LIMIT 10;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.203 seconds)
OK
No rows affected (2.095 seconds)
No Stats for movielens@ratings, Columns: rating, movieid
No Stats for movielens@ratings, Columns: movieid
No Stats for movielens@movienames, Columns: movieid, title
Query ID = jovyan_20201014001447_5f298913-c9fc-472b-a0e6-4c9e279aef20
Total jobs = 9
Launching Job 1 out of 9
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1602633141551_0006, Tracking URL = http://jupyter-thedatasociety-2dlab-2dhadoop-2dbyh3hjxu:8088/proxy/application_1602633

What do you think about this result??

#Select to see the result

<font color='white'>
SELECT movieID, AVG(rating) as ratingAVG<br>
FROM ratings<br>
GROUP BY movieID<br>
ORDER BY ratingAVG DESC<br>
LIMIT 10;
</font>


## Find the movie with the highest average time, only considering movies with more than 10 ratings

#Select to see the result

<font color='white'>
SOLUTION 1: <br>
SELECT r.movieID, m.title, AVG(r.rating) as ratingAVG, COUNT(r.rating) as ratingCount<br>
FROM ratings r JOIN movieNames m ON r.movieID = m.movieID<br>
GROUP BY r.movieID, m.title<br>
HAVING ratingCount > 10<br>
ORDER BY ratingAVG DESC<br>
LIMIT 10;<br><br>
SOLUTION 2:<br>
CREATE VIEW avgMoviesIds AS<br>
SELECT movieID, AVG(rating) as ratingAVG<br>
FROM ratings<br>
GROUP BY movieID<br>
ORDER BY ratingAVG DESC;<br>
<br>
SELECT n.title as titles, a.ratingAVG as average, t.ratingCount as rating<br>
FROM avgMoviesIds a JOIN topMoviesIds t ON a.movieID = t.movieID<br>
    JOIN movieNames n ON t.movieID = n.movieID<br>
WHERE t.ratingCount >10<br>
ORDER BY average DESC<br>
LIMIT 10;
</font>
